# CLEANING

![alt text](https://miro.medium.com/max/413/0*Cir0TzUEkHMbb8QB "Cleaning data")

- Cargamos **librerías, funciones y el data set** que vamos a usar para la limpieza 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [6]:
#ORDENAR - esta es la importacion del archivo de las funciones de limpieza que me he creado
import sys
sys.path.append("src/")
from src.cleaning_functions import *

In [7]:
ds = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

- Hacemos una copia del data set "just in case"...

In [8]:
df = ds.copy()

# EXPLORACIÓN DATA SET

![alt text](https://memegenerator.net/img/instances/40379228/let-me-take-a-look-at-this.jpg "Take a look")

In [9]:
df.shape #25.723 Lineas y 24 columnas

(25723, 24)

In [11]:
df.sample()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
8588,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.info

<bound method DataFrame.info of       Case Number         Date    Year        Type    Country  \
0      2018.06.25  25-Jun-2018  2018.0     Boating        USA   
1      2018.06.18  18-Jun-2018  2018.0  Unprovoked        USA   
2      2018.06.09  09-Jun-2018  2018.0     Invalid        USA   
3      2018.06.08  08-Jun-2018  2018.0  Unprovoked  AUSTRALIA   
4      2018.06.04  04-Jun-2018  2018.0    Provoked     MEXICO   
...           ...          ...     ...         ...        ...   
25718         NaN          NaN     NaN         NaN        NaN   
25719         NaN          NaN     NaN         NaN        NaN   
25720         NaN          NaN     NaN         NaN        NaN   
25721         NaN          NaN     NaN         NaN        NaN   
25722          xx          NaN     NaN         NaN        NaN   

                  Area                        Location     Activity  \
0           California     Oceanside, San Diego County     Paddling   
1              Georgia  St. Simon Island, Gly

In [17]:
df.describe().T #Solo hay dos columnas creadas como numéricas

,count,mean,std,min,25%,50%,75%,max
Year,6300.0,1927.272381,281.116308,0.0,1942.0,1977.0,2005.0,2018.0
original order,6309.0,3155.999683,1821.396206,2.0,1579.0,3156.0,4733.0,6310.0


In [30]:
df.isna().sum().sort_values(ascending = False) #NaN everiwhere

Unnamed: 22               25722
Unnamed: 23               25721
Time                      22775
Species                   22259
Age                       22252
Sex                       19986
Activity                  19965
Location                  19961
Fatal (Y/N)               19960
Area                      19876
Name                      19631
Country                   19471
Injury                    19449
Investigator or Source    19438
Type                      19425
Year                      19423
href formula              19422
Date                      19421
pdf                       19421
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Case Number               17021
dtype: int64

df.isna().sum()

Hay muchas columnas bastante sucias donde prácticamente todo son NaN.
De hecho "Unnamed 22 y 23" solo tienen dos datos

In [ ]:
df[["Unnamed: 22","Unnamed: 23"]].notna().sum()

- Podemos ver el porcentaje de NaN por cada columna para ver su grado de suciedad/validez:

In [ ]:
df.isnull().sum().apply(lambda x: x/df.shape[0]).sort_values(ascending=False)

In [ ]:
df.loc[df['Unnamed: 22'].notna()] #stopped here, mierdato

In [ ]:
df.loc[df['Unnamed: 23'].notna()] #Teramo y change filename

In [ ]:
df.columns

- Vemos que hay un par de columnsa con espacios que podemos quitar: Species y Sex

In [ ]:
df.columns = df.columns.str.rstrip()

In [ ]:
df.columns

# LIMPIEZA

![alt text](https://miro.medium.com/max/568/1*S1HH5F8PqWWcId9sb0L8og.jpeg "Dropna")

- Quitamos todas las filas y columnas que tengan todo NaN

In [ ]:
df.dropna(axis = 0, how = 'all', inplace = True)
df.dropna(axis = 1, how = 'all', inplace = True)

df.drop_duplicates(inplace=True)


df.shape #hemos quitado bastantes filas, realmente columnas no hemos limpiado directamente ya que Unnamed 22 y 23 tenian 1 y 2 datos

In [ ]:
df["Case Number"].sample(40)

In [ ]:
df[["Case Number", "Date","Investigator or Source",'Case Number.1', 'Case Number.2', 'original order']].sample(30)

- Echando una ojeada a estas columnas podemosdescartarlas ya que o están duplicadas o son datos que no nos van a aportar para nuestras hipótesis

In [ ]:
df[["pdf", "href","href formula"]].sample(20)

- Miramos uno de los links del href para ver contenido:

In [ ]:
df["href"][3513] 

https://sharkattackfile.net/spreadsheets/pdf_directory/1967.08.25-Casucci.pdf

In [ ]:
df["href formula"][6091]

http://sharkattackfile.net/spreadsheets/pdf_directory/1830.04.30-Bromwick.pdf


- Algunos estaban vacíos (iamgino que habrá cambiado en algo la dirección), otros contienen pdf´s con datos del ataque reportado: imagenes, recortes de periódico, fotos de la ubicación, etc....
Pero en este caso no voy a utilizarlos.

- Ojeadas las columnas podemos hacer **drop** de las que **no considero interesantes** para el caso:

'Case Number','Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22','Unnamed: 23'

In [ ]:
df.drop(['Case Number','Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22','Unnamed: 23'], axis=1, inplace=True)

In [ ]:
df.columns

### YEAR y DATE COLUMN

In [ ]:
df.Year.unique() #Vamos a ver si podemos rescatar algun año a través del contenido de la columna date

- Vemos años un poco raros, así que paso a verlos en detalle

In [ ]:
df[(df.Year == 0)]

- Hay 125 lineas con el df.Year == CERO pero en df.Date si que figuran datos, a ver que podemos rescatar de ahí.

In [ ]:
list(df.Date.unique())

- Mucho reported y mucho espacio en blanco:

In [ ]:
df.Date = df.Date.replace(regex=r'(?i)Reported\s{1,9}',value='')
list(df.Date.unique())

- Aunque no salen en los **uniques** hay muchos Dates que salen como rangos, o datos de antes de Cristo.
- Vamos a denominarlos momentáneamente salvables para ver cuantos hay exactamente:

In [ ]:
salvables = df.loc[(df["Year"] == 0) & (df["Date"] != np.nan)]
salvables.shape

In [ ]:
salvables.sample(30)

- Llamamos a las funciones definidas en el **cleanin_functions.py**, mas concretamente a rescatar fechas que aplica 3 funciones secuencialmente donde coge por orden los que contienen BC, los que son fechas sueltas (tipo Before YYYY) y luego los intervalos de los cuales saca la media, para rellenar los datos de df.Year para esos valores de df.Dates  #TODO, mira tema de late´s y demas para meterlos tb

In [ ]:
df.Year = df.Date.apply(rescatar_fechas)

- Si miramos el dato primero de nuestra lista anterior vemos que:

In [ ]:
df["Date"][6228]

In [ ]:
df["Year"][6228]

In [ ]:
df["Date"][6265]

In [ ]:
df["Year"][6265]

In [ ]:
#pd.set_option('max_rows', None)

In [ ]:
df[["Date","Year"]].sample(10)

In [ ]:
#pd.set_option('max_rows', 20)

## AREA / LOCATION COLUMNS

- Ambas columnas son bastante imprecisas en cuanto a ubicacion, por lo que mejor voy a trabajar sobre la columna **country.**

In [ ]:
df[["Country","Area","Location"]].sample(10)

- Country voy a limpiar con una funcion metidad en cleaning_functios.py llamada paises. He cogido una lista de paises de Github que esta
metida en un CSV, lo que hace la funcion es mirar si la cadena correspondiente está en el CSV y sino le asigna NaN adema de un par de incorrecciones de la escritura de nombres que tambien he metido

In [ ]:
df.Country = df.Country.apply(paises)

In [ ]:
df.Country.notna().sum()

In [ ]:
#Esto se puede mejorar, para resultados sea...

# ACTIVITY COLUMN

In [ ]:
df.Activity.sample(30)

- La lista de actividades es amplia, vamos a agruparlas y filtrarlas a través de una función llamada actividad tambien contenida en cleaning_functions.py

In [ ]:
df.Activity = df.Activity.apply(actividad)
df.Activity.sample(30)

# COLUMNA INJURY

In [ ]:
list(df["Fatal (Y/N)"].unique())

In [ ]:
df[["Fatal (Y/N)","Injury"]].sample(30)

- El tema de valores distintos de los previsibles "Y"/ "N" ademas de categorizar la columna injury lo vamos a hacer a través de dos funciones

In [ ]:
df["Injury"] = df["Injury"].apply(lesiones)

In [ ]:
df['Fatal (Y/N)'] = df["Fatal (Y/N)"].apply(fatal)

In [ ]:
df[["Fatal (Y/N)","Injury"]].sample(10)

In [ ]:
#TODO age

# SPECIES COLUMN

In [ ]:
df.Species.sample(30)

- Para limpiar esto vamos a usar algo parecido a lo que hemos hecho con activity para categorizar y leer las cadenas de dentro.Usaré la lista de nombres de https://sharkattackfile.net/species.htm
 que es de donde viene nuestro dataframe además

In [ ]:
df.Species = df.Species.apply(species)

In [ ]:
df.Species.sample(30)

# Exportamos el data frame limpio a un CSV nuevo

In [ ]:
df.to_csv("src/attack_limpio.csv",index=False)

- La visualizacion continua en `analysis.ipynb` [📑](analysis.ipynb) 